In [ ]:
!pip install pytesseract
!sudo apt-get update
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev

In [ ]:
! pip install python-docx

In [ ]:
! pip install pdf2image -q

In [ ]:
import os
from pdf2image import convert_from_path
from docx import Document
import subprocess

In [ ]:
import pytesseract
from PIL import Image

In [ ]:
!apt-get install poppler-utils

In [ ]:

def extract_text_from_pdf(file_path):
    text = ""
    images = convert_from_path(file_path)
    for image in images:
        text += pytesseract.image_to_string(image)
    return text

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    return '\n'.join(text)

def extract_text_from_doc(file_path):
    output_file_path = file_path + ".txt"
    subprocess.run(['antiword', file_path, '-m', 'UTF-8', '-t', output_file_path])
    with open(output_file_path, 'r') as file:
        text = file.read()
    return text

def extract_text(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == '.pdf':
        return extract_text_from_pdf(file_path)
    elif ext == '.docx':
        return extract_text_from_docx(file_path)
    elif ext == '.doc':
        return extract_text_from_doc(file_path)
    else:
        raise ValueError("Unsupported file format")

# Example usage:
# file_path = "D:/new/itsolera-assignment-1/CV.pdf"

file_path = input("enter your file")
# Change to your file path
text = extract_text(file_path)
print(text)

In [ ]:
import spacy
import re
from spacy.matcher import Matcher

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

def preprocess_text(text):
    text = ' '.join(text.split())
    text = re.sub(r'[\r\n]+', '\n', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def split_section_content(content):
    """ Split section content using common delimiters and heuristics. """
    return re.split(r'\s*[,;]\s*|\n|(?<=\S)\s{2,}(?=\S)', content)

def extract_info_from_cv(raw_text):
    text = preprocess_text(raw_text)
    doc = nlp(text)

    matcher = Matcher(nlp.vocab)

    patterns = {
        'SKILLS_HEADER': [
            [{'LOWER': 'skills'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}],
            [{'LOWER': 'abilities'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}],
            [{'LOWER': 'expertise'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}]
        ],
        'CERTIFICATES_HEADER': [
            [{'LOWER': 'certifications'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}],
            [{'LOWER': 'certificates'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}],
            [{'LOWER': 'accreditations'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}]
        ],
        'PROJECTS_HEADER': [
            [{'LOWER': 'projects'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}],
            [{'LOWER': 'works'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}],
            [{'LOWER': 'assignments'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}]
        ],
        'EXPERIENCE_HEADER': [
            [{'LOWER': 'experience'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}],
            [{'LOWER': 'employment'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}],
            [{'LOWER': 'work history'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}]
        ],
        'EDUCATION_HEADER': [
            [{'LOWER': 'education'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}],
            [{'LOWER': 'academic background'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}],
            [{'LOWER': 'qualifications'}, {'IS_PUNCT': True, 'OP': '?'}, {'IS_SPACE': True, 'OP': '*'}]
        ],
    }

    for label, patterns_list in patterns.items():
        for pattern in patterns_list:
            matcher.add(label, [pattern])

    matches = matcher(doc)

    extracted_info = {'name': '', 'email': '', 'phone': '', 'skills': [], 'certificates': [], 'projects': [], 'experience': [], 'education': []}

    # Extract the name (first two proper nouns)
    proper_nouns = [token.text for token in doc if token.pos_ == 'PROPN']
    if len(proper_nouns) >= 2:
        extracted_info['name'] = ' '.join(proper_nouns[:2])

    # Extract email and phone number using regex
    email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
    phone_pattern = re.compile(r'\b(?:\d{10}|\d{3}[-.\s]\d{3}[-.\s]\d{4}|\d{4}[-.\s]?\d{7}|\d{11})\b')

    email_match = email_pattern.search(text)
    phone_match = phone_pattern.search(text)

    if email_match:
        extracted_info['email'] = email_match.group().strip()
    if phone_match:
        extracted_info['phone'] = phone_match.group().strip()

    # Extract sections based on headers
    section_starts = {}
    for match_id, start, end in matches:
        label = nlp.vocab.strings[match_id]
        section_starts[label] = start

    sorted_sections = sorted(section_starts.items(), key=lambda x: x[1])

    sections = {
        'skills': [],
        'certificates': [],
        'projects': [],
        'experience': [],
        'education': []
    }

    for i, (section, start) in enumerate(sorted_sections):
        end = sorted_sections[i + 1][1] if i + 1 < len(sorted_sections) else len(doc)
        content = doc[start:end].text.strip()
        items = split_section_content(content)
        if section == 'SKILLS_HEADER':
            sections['skills'] = [item for item in items if item.strip() and not re.match(r'^(Certifications|Projects|Experience|Education):', item, re.IGNORECASE)]
        elif section == 'CERTIFICATES_HEADER':
            sections['certificates'] = [item for item in items if item.strip() and not re.match(r'^(Projects|Experience|Education):', item, re.IGNORECASE)]
        elif section == 'PROJECTS_HEADER':
            sections['projects'] = [item for item in items if item.strip() and not re.match(r'^(Experience|Education):', item, re.IGNORECASE)]
        elif section == 'EXPERIENCE_HEADER':
            sections['experience'] = [item for item in items if item.strip() and not re.match(r'^Education:', item, re.IGNORECASE)]
        elif section == 'EDUCATION_HEADER':
            sections['education'] = [item for item in items if item.strip() and not re.match(r'^(Skills|Certifications|Projects|Experience):', item, re.IGNORECASE)]

    # Remove section headers from the first element of each section
    for key in sections:
        if sections[key] and re.match(r'^(Skills|Certifications|Projects|Experience|Education):', sections[key][0], re.IGNORECASE):
            sections[key][0] = re.sub(r'^(Skills|Certifications|Projects|Experience|Education):\s*', '', sections[key][0], flags=re.IGNORECASE)

    extracted_info['skills'] = sections['skills']
    extracted_info['certificates'] = sections['certificates']
    extracted_info['projects'] = sections['projects']
    extracted_info['experience'] = sections['experience']
    extracted_info['education'] = sections['education']

    return extracted_info

# Sample OCR extracted text from a CV
cv_text = """
Maryam Murtaza
AI Engineer

Email: maryammurtazamughal@gmail.com
Phone: 0349-2261022

Skills: Python, Machine Learning,Computer Vision, Data Analysis

Projects:
Developed an end-to-end Diabetic Retinopathy Detection Software using Computer Vision
Created a web application for real-time data visualization

Experience:
Software Engineer at ABC Corp (2018-2022)
Data Analyst at XYZ Ltd (2015-2018)

Education:
BSc in Computer Science, XYZ University (2011-2015)
"""

# Extract information from the sample text
extracted_info = extract_info_from_cv(cv_text)
print(extracted_info)